# Tag 3: Navigation & Umgebungserkennung
## Unitree G1 Humanoid Roboter Academy

### Agenda Tag 3:
1. SLAM (Simultaneous Localization and Mapping)
2. LiDAR-Integration (Livox MID-360)
3. Mapping & Lokalisierung
4. Globale Pfadplanung (A*, RRT)
5. Lokale Hindernisvermeidung
6. Workshop: Navigationsaufgabe mit Hindernisfeld
7. Sensorintegration (RealSense)

---
## 1. SLAM Grundlagen

### 1.1 Was ist SLAM?

**SLAM = Simultaneous Localization and Mapping**

**Problem:**
- Roboter in unbekannter Umgebung
- Keine Karte verfügbar
- Keine genaue Position bekannt

**Lösung:**
- **Gleichzeitig:**
  - Karte der Umgebung erstellen (Mapping)
  - Eigene Position in der Karte bestimmen (Localization)

```
┌────────────────────────────────────┐
│  Sensor Data (LiDAR, RGB-D)        │
└──────────────┬─────────────────────┘
               │
┌──────────────▼─────────────────────┐
│  SLAM Algorithm                    │
│  - Feature Extraction              │
│  - Data Association                │
│  - State Estimation                │
│  - Map Update                      │
└──────────┬───────────┬─────────────┘
           │           │
  ┌────────▼───┐  ┌───▼────────┐
  │ Position   │  │ Map        │
  │ (x, y, θ)  │  │ (3D Cloud) │
  └────────────┘  └────────────┘
```

### 1.2 KISS-ICP SLAM

Das G1-Repository nutzt **KISS-ICP** (Keep It Small and Simple - Iterative Closest Point):

**Vorteile:**
- ✓ Echtzeit-fähig (20+ Hz)
- ✓ Keine Loop-Closure benötigt
- ✓ Robust gegen Drift
- ✓ Python-Implementation
- ✓ Funktioniert mit beliebigem 3D LiDAR

**Algorithmus:**
1. Empfange neuen LiDAR-Scan
2. Voxel-Downsampling (Reduktion Punktanzahl)
3. ICP-Matching mit vorheriger Karte
4. Schätze Transformation (Δx, Δy, Δθ)
5. Update Roboter-Pose
6. Füge Scan zur Karte hinzu

---
## 2. LiDAR-Integration: Livox MID-360

### 2.1 Livox MID-360 Spezifikationen

| Spezifikation | Wert |
|---------------|------|
| **Sichtfeld (FoV)** | 360° horizontal, ±59° vertikal |
| **Reichweite** | 0.05 - 70 m (bei 10% reflectivity) |
| **Punktrate** | 200,000 pts/s |
| **Genauigkeit** | < 2 cm (@ 20m) |
| **Scan-Pattern** | Non-repetitive rosette |
| **Interface** | Ethernet (UDP) |
| **IP** | 192.168.1.1 (default) |

### 2.2 LiDAR Setup & Konfiguration

**Netzwerk-Konfiguration:**

1. Verbinden Sie den MID-360 via Ethernet
2. Setzen Sie Ihren PC auf statische IP: `192.168.1.50`
3. Konfigurieren Sie `mid360_config.json`:

In [ ]:
# Beispiel: mid360_config.json
config_example = {
    "MID360": {
        "lidar_net_info": {
            "cmd_data_port": 56100,
            "push_msg_port": 56200,
            "point_data_port": 56300,
            "imu_data_port": 56400,
            "log_data_port": 56500
        },
        "host_net_info": {
            "cmd_data_ip": "192.168.1.50",  # Ihre PC-IP!
            "cmd_data_port": 56101,
            "push_msg_ip": "192.168.1.50",
            "push_msg_port": 56201,
            "point_data_ip": "192.168.1.50",
            "point_data_port": 56301,
            "imu_data_ip": "192.168.1.50",
            "imu_data_port": 56401,
            "log_data_ip": "192.168.1.50",
            "log_data_port": 56501
        }
    }
}

import json
print("Beispiel-Konfiguration für Livox MID-360:")
print(json.dumps(config_example, indent=2))

### 2.3 Live Point-Cloud Viewing

In [ ]:
# Setup für LiDAR-Visualisierung
import sys
sys.path.append('../unitree_g1_vibes')

# Importiere LiDAR-Wrapper
# from livox2_python import LivoxLidar

print("LiDAR-Treiber verfügbar!")
print("\nStarten mit:")
print("cd ../unitree_g1_vibes")
print("python3 live_points.py")
print("\nDies öffnet Open3D-Fenster mit Live-Point-Cloud")

**Was passiert in `live_points.py`:**

```python
# Vereinfachte Version:
import open3d as o3d
from livox2_python import LivoxLidar

# Initialize LiDAR
lidar = LivoxLidar(config_file="mid360_config.json")
lidar.start()

# Open3D Visualizer
vis = o3d.visualization.Visualizer()
vis.create_window("Livox MID-360 Live View")

pcd = o3d.geometry.PointCloud()
vis.add_geometry(pcd)

while True:
    # Empfange neuen Frame
    points = lidar.get_latest_points()  # Nx3 array
    
    # Update Point-Cloud
    pcd.points = o3d.utility.Vector3dVector(points)
    pcd.paint_uniform_color([0.5, 0.5, 1.0])  # Blau
    
    # Render
    vis.update_geometry(pcd)
    vis.poll_events()
    vis.update_renderer()
```

---
## 3. SLAM mit KISS-ICP

### 3.1 SLAM Starten

In [ ]:
# SLAM mit KISS-ICP starten
# Im Terminal:
# cd ../unitree_g1_vibes
# python3 live_slam.py

print("SLAM-Visualisierung:")
print("- Grüne Punkte: Neue Scans")
print("- Weiße Punkte: Karte (akkumuliert)")
print("- Rote Linie: Trajektorie (Roboter-Pfad)")
print("- Koordinatensystem: Aktuelle Pose")

### 3.2 SLAM-Daten extrahieren

**Pose-Tracking:**

In [ ]:
# Beispiel: SLAM-Daten auslesen
# (Pseudo-Code basierend auf live_slam.py)

from kiss_icp.kiss_icp import KissICP
import numpy as np

# Initialize SLAM
# slam = KissICP()

# Process new scan
# points = lidar.get_latest_points()  # Nx3 numpy array
# pose = slam.register_frame(points)  # 4x4 transformation matrix

# Extrahiere Position
def extract_position_2d(pose_matrix):
    """
    Extrahiert (x, y, theta) aus 4x4 Transformationsmatrix
    """
    x = pose_matrix[0, 3]
    y = pose_matrix[1, 3]
    
    # Rotation um Z-Achse
    theta = np.arctan2(pose_matrix[1, 0], pose_matrix[0, 0])
    
    return x, y, theta

# Beispiel:
example_pose = np.eye(4)
example_pose[0, 3] = 1.5  # x = 1.5 m
example_pose[1, 3] = 0.8  # y = 0.8 m

x, y, theta = extract_position_2d(example_pose)
print(f"Position: x={x:.2f}m, y={y:.2f}m, theta={np.degrees(theta):.1f}°")

### 3.3 2D Occupancy Grid erstellen

Für Navigation benötigen wir eine **2D-Karte** (Occupancy Grid):

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

def create_occupancy_grid(point_cloud, resolution=0.1, size=20.0):
    """
    Erzeugt 2D Occupancy Grid aus 3D Point-Cloud
    
    point_cloud: Nx3 numpy array (x, y, z)
    resolution: Grid-Auflösung in Metern
    size: Kartengröße (size x size Meter)
    
    Returns: 2D array (occupied=1, free=0, unknown=-1)
    """
    grid_size = int(size / resolution)
    grid = np.zeros((grid_size, grid_size), dtype=np.int8)
    grid[:] = -1  # Initialisiere als unbekannt
    
    # Filter: Nur Punkte nahe Bodenhöhe (z < 2m)
    ground_points = point_cloud[point_cloud[:, 2] < 2.0]
    
    # Konvertiere zu Grid-Koordinaten
    center = grid_size // 2
    for point in ground_points:
        x, y, z = point
        
        # Grid-Indizes
        i = int(center + x / resolution)
        j = int(center + y / resolution)
        
        # Bounds-Check
        if 0 <= i < grid_size and 0 <= j < grid_size:
            grid[j, i] = 1  # Belegt
    
    return grid

# Test mit synthetischen Daten:
# Simuliere Raum mit Wänden
test_points = []

# Wand rechts (x=5m)
for y in np.linspace(-5, 5, 100):
    test_points.append([5.0, y, 0.5])

# Wand links (x=-5m)
for y in np.linspace(-5, 5, 100):
    test_points.append([-5.0, y, 0.5])

# Hindernis in der Mitte
for x in np.linspace(-1, 1, 50):
    for y in np.linspace(-1, 1, 50):
        test_points.append([x, y, 0.8])

test_cloud = np.array(test_points)

# Erstelle Grid
grid = create_occupancy_grid(test_cloud, resolution=0.1, size=12.0)

# Visualisiere
plt.figure(figsize=(10, 10))
plt.imshow(grid, cmap='gray_r', origin='lower', extent=[-6, 6, -6, 6])
plt.colorbar(label='Occupancy (1=belegt, 0=frei, -1=unbekannt)')
plt.xlabel('X [m]')
plt.ylabel('Y [m]')
plt.title('2D Occupancy Grid aus LiDAR-Daten')
plt.grid(True, alpha=0.3)
plt.show()

print(f"Grid-Größe: {grid.shape}")
print(f"Belegte Zellen: {np.sum(grid == 1)}")

---
## 4. Globale Pfadplanung

### 4.1 A* Algorithmus

**A*** ist der Standard für Grid-basierte Pfadplanung:

**Algorithmus:**
1. Start: Open-Liste = {Start-Knoten}, Closed-Liste = {}
2. Wähle Knoten mit kleinstem f(n) = g(n) + h(n) aus Open-Liste
   - g(n) = Kosten von Start zu n
   - h(n) = Heuristik (geschätzte Kosten von n zu Ziel)
3. Wenn Ziel erreicht → Rekonstruiere Pfad
4. Expandiere Nachbarn, update Open-/Closed-Listen
5. Wiederhole ab 2.

In [ ]:
import heapq
import numpy as np

def astar_planning(grid, start, goal):
    """
    A* Pfadplanung auf 2D Occupancy Grid
    
    grid: 2D numpy array (1=belegt, 0=frei)
    start: (x, y) tuple
    goal: (x, y) tuple
    
    Returns: Liste von (x, y) Wegpunkten
    """
    def heuristic(a, b):
        # Euklidische Distanz
        return np.sqrt((a[0] - b[0])**2 + (a[1] - b[1])**2)
    
    def get_neighbors(pos):
        # 8-Konnektivität
        directions = [
            (1, 0), (-1, 0), (0, 1), (0, -1),  # Cardinal
            (1, 1), (1, -1), (-1, 1), (-1, -1)  # Diagonal
        ]
        neighbors = []
        for dx, dy in directions:
            nx, ny = pos[0] + dx, pos[1] + dy
            # Check bounds und Hindernisse
            if (0 <= nx < grid.shape[1] and 
                0 <= ny < grid.shape[0] and 
                grid[ny, nx] != 1):  # Nicht belegt
                cost = 1.414 if dx != 0 and dy != 0 else 1.0  # Diagonal teurer
                neighbors.append(((nx, ny), cost))
        return neighbors
    
    # Initialisierung
    open_set = []
    heapq.heappush(open_set, (0, start))
    came_from = {}
    g_score = {start: 0}
    f_score = {start: heuristic(start, goal)}
    
    while open_set:
        current_f, current = heapq.heappop(open_set)
        
        # Ziel erreicht?
        if current == goal:
            # Rekonstruiere Pfad
            path = [current]
            while current in came_from:
                current = came_from[current]
                path.append(current)
            path.reverse()
            return path
        
        # Expandiere Nachbarn
        for neighbor, cost in get_neighbors(current):
            tentative_g = g_score[current] + cost
            
            if neighbor not in g_score or tentative_g < g_score[neighbor]:
                came_from[neighbor] = current
                g_score[neighbor] = tentative_g
                f_score[neighbor] = tentative_g + heuristic(neighbor, goal)
                heapq.heappush(open_set, (f_score[neighbor], neighbor))
    
    return None  # Kein Pfad gefunden

# Test:
start_pos = (10, 10)
goal_pos = (110, 110)

path = astar_planning(grid, start_pos, goal_pos)

if path:
    print(f"✓ Pfad gefunden mit {len(path)} Wegpunkten")
    
    # Visualisiere
    plt.figure(figsize=(10, 10))
    plt.imshow(grid, cmap='gray_r', origin='lower', extent=[-6, 6, -6, 6])
    
    # Konvertiere Pfad zu Welt-Koordinaten
    resolution = 0.1
    center = grid.shape[0] // 2
    path_world = [(x - center) * resolution for x, y in path]
    path_world_y = [(y - center) * resolution for x, y in path]
    
    plt.plot(path_world, path_world_y, 'r-', linewidth=2, label='A* Pfad')
    plt.plot(path_world[0], path_world_y[0], 'go', markersize=10, label='Start')
    plt.plot(path_world[-1], path_world_y[-1], 'ro', markersize=10, label='Ziel')
    
    plt.xlabel('X [m]')
    plt.ylabel('Y [m]')
    plt.title('A* Pfadplanung')
    plt.legend()
    plt.grid(True, alpha=0.3)
    plt.show()
else:
    print("✗ Kein Pfad gefunden!")

### 4.2 RRT (Rapidly-exploring Random Tree)

**Alternative zu A*:** Besser für hochdimensionale Räume und komplexe Hindernisse.

**Algorithmus:**
1. Baum initialisieren mit Start-Knoten
2. Wiederhole N mal:
   - Sample zufälligen Punkt
   - Finde nächsten Baum-Knoten
   - Erweitere Baum in Richtung Sample
   - Prüfe Kollision
3. Wenn Ziel erreicht → Extrahiere Pfad

In [ ]:
# Simplified RRT implementation
def rrt_planning(grid, start, goal, max_iterations=1000, step_size=5):
    """
    RRT Pfadplanung
    
    grid: 2D numpy array
    start, goal: (x, y) tuples
    max_iterations: Maximale Anzahl Iterationen
    step_size: Schrittgröße beim Erweitern
    """
    class Node:
        def __init__(self, pos, parent=None):
            self.pos = pos
            self.parent = parent
    
    def distance(p1, p2):
        return np.sqrt((p1[0] - p2[0])**2 + (p1[1] - p2[1])**2)
    
    def is_collision_free(p1, p2):
        # Line-of-sight check
        n_steps = int(distance(p1, p2) / 1.0)
        for i in range(n_steps + 1):
            alpha = i / max(n_steps, 1)
            x = int(p1[0] + alpha * (p2[0] - p1[0]))
            y = int(p1[1] + alpha * (p2[1] - p1[1]))
            if not (0 <= x < grid.shape[1] and 0 <= y < grid.shape[0]):
                return False
            if grid[y, x] == 1:
                return False
        return True
    
    # Initialize
    tree = [Node(start)]
    
    for iteration in range(max_iterations):
        # Sample random point (90%) oder Ziel (10%)
        if np.random.rand() < 0.1:
            sample = goal
        else:
            sample = (
                np.random.randint(0, grid.shape[1]),
                np.random.randint(0, grid.shape[0])
            )
        
        # Find nearest node
        nearest_node = min(tree, key=lambda n: distance(n.pos, sample))
        
        # Steer towards sample
        direction = np.array(sample) - np.array(nearest_node.pos)
        length = np.linalg.norm(direction)
        if length > step_size:
            direction = direction / length * step_size
        
        new_pos = tuple((np.array(nearest_node.pos) + direction).astype(int))
        
        # Check collision
        if is_collision_free(nearest_node.pos, new_pos):
            new_node = Node(new_pos, nearest_node)
            tree.append(new_node)
            
            # Check if goal reached
            if distance(new_pos, goal) < step_size:
                # Reconstruct path
                path = []
                current = new_node
                while current:
                    path.append(current.pos)
                    current = current.parent
                path.reverse()
                return path
    
    return None  # No path found

print("RRT-Funktion definiert.")
print("Test mit: rrt_planning(grid, start_pos, goal_pos)")

---
## 5. Lokale Hindernisvermeidung

### 5.1 Dynamic Window Approach (DWA)

**Problem:** Globaler Pfad berücksichtigt keine dynamischen Hindernisse.

**Lösung: DWA** - Lokale Trajektorien-Optimierung

**Algorithmus:**
1. Berechne "Dynamic Window" - erlaubte (vx, vy, ω) basierend auf:
   - Maximaler Beschleunigung
   - Aktueller Geschwindigkeit
2. Sample Trajektorien in diesem Window
3. Bewerte jede Trajektorie nach:
   - Distanz zu Hindernissen
   - Fortschritt zum Ziel
   - Geschwindigkeit (höher = besser)
4. Wähle beste Trajektorie

In [ ]:
# Simplified DWA
def dynamic_window_approach(current_vel, goal_pos, obstacles, dt=0.1):
    """
    DWA für lokale Hindernisvermeidung
    
    current_vel: (vx, vy, omega) aktuell
    goal_pos: (x, y) Zielposition
    obstacles: Liste von (x, y) Hindernispositionen
    dt: Zeitschritt
    
    Returns: (vx, vy, omega) beste Geschwindigkeit
    """
    # Grenzen
    max_v = 0.6  # m/s
    max_omega = 0.6  # rad/s
    max_accel = 1.0  # m/s²
    max_alpha = 1.0  # rad/s²
    
    # Dynamic Window
    vx_min = max(-max_v, current_vel[0] - max_accel * dt)
    vx_max = min(max_v, current_vel[0] + max_accel * dt)
    omega_min = max(-max_omega, current_vel[2] - max_alpha * dt)
    omega_max = min(max_omega, current_vel[2] + max_alpha * dt)
    
    # Sample Trajektorien
    best_score = -np.inf
    best_vel = current_vel
    
    for vx in np.linspace(vx_min, vx_max, 10):
        for omega in np.linspace(omega_min, omega_max, 10):
            # Simuliere Trajektorie (1 Sekunde)
            sim_time = 1.0
            sim_x, sim_y, sim_theta = 0, 0, 0
            
            for t in np.arange(0, sim_time, dt):
                sim_x += vx * np.cos(sim_theta) * dt
                sim_y += vx * np.sin(sim_theta) * dt
                sim_theta += omega * dt
            
            # Evaluate
            # 1. Distanz zum Ziel
            goal_dist = np.sqrt((goal_pos[0] - sim_x)**2 + (goal_pos[1] - sim_y)**2)
            goal_score = 1.0 / (1.0 + goal_dist)
            
            # 2. Hindernisdistanz (Mindestabstand)
            min_obstacle_dist = np.inf
            for obs_x, obs_y in obstacles:
                dist = np.sqrt((obs_x - sim_x)**2 + (obs_y - sim_y)**2)
                min_obstacle_dist = min(min_obstacle_dist, dist)
            
            if min_obstacle_dist < 0.3:  # Zu nah!
                obstacle_score = -10.0
            else:
                obstacle_score = min_obstacle_dist
            
            # 3. Geschwindigkeits-Bonus
            velocity_score = abs(vx) / max_v
            
            # Gesamt-Score
            total_score = (
                2.0 * goal_score +
                1.0 * obstacle_score +
                0.5 * velocity_score
            )
            
            if total_score > best_score:
                best_score = total_score
                best_vel = (vx, 0.0, omega)
    
    return best_vel

# Test:
current_vel = (0.2, 0.0, 0.0)
goal_pos = (5.0, 0.0)
obstacles = [(2.0, 0.5), (3.0, -0.5)]

best_vel = dynamic_window_approach(current_vel, goal_pos, obstacles)
print(f"Beste Geschwindigkeit: vx={best_vel[0]:.2f} m/s, omega={best_vel[2]:.2f} rad/s")

---
## 6. RealSense Integration (RGB-D)

### 6.1 RealSense D435i Übersicht

Zusätzlich zum LiDAR hat der G1 eine **Intel RealSense D435i** Kamera:

| Spezifikation | Wert |
|---------------|------|
| **RGB** | 1920×1080 @ 30 FPS |
| **Tiefe** | 1280×720 @ 30 FPS |
| **Reichweite** | 0.3 - 10 m |
| **FoV** | 69° H × 42° V (Tiefe) |
| **IMU** | Accel + Gyro @ 200 Hz |

### 6.2 RealSense Streaming Setup

Das Repository nutzt **GStreamer** für low-latency Streaming:

**Auf Jetson (G1):**
```bash
python3 jetson_realsense_stream.py --client-ip <PC_IP>
```

**Auf PC:**
```bash
python3 receive_realsense_gst.py
```

**Ports:**
- 5600: RGB (H.264 encoded)
- 5602: Depth Colormap (H.264 encoded)

---
## 7. Workshop: Navigationsaufgabe mit Hindernisfeld

### Aufgabe: Navigiere durch Labyrinth

**Szenario:**
- Roboter startet bei (0, 0)
- Ziel: (8, 8)
- Hindernisse verteilt in Umgebung

**Schritte:**
1. Erstelle SLAM-Karte mit LiDAR
2. Plane globalen Pfad mit A*
3. Folge Pfad mit lokalem Controller (DWA)
4. Erreiche Ziel

In [ ]:
# Vollständiges Navigations-Beispiel (Pseudo-Code)

def navigate_to_goal(bot, slam, goal_pos):
    """
    Vollständige Navigation zu Zielpunkt
    
    bot: LocoClient Instanz
    slam: KISS-ICP SLAM Instanz
    goal_pos: (x, y) Zielposition
    """
    print(f"Navigiere zu Ziel: {goal_pos}")
    
    # 1. Erstelle Karte
    print("Scanne Umgebung...")
    # point_cloud = slam.get_map()
    # grid = create_occupancy_grid(point_cloud)
    
    # 2. Plane globalen Pfad
    # current_pose = slam.get_pose()
    # start_grid = world_to_grid(current_pose[:2])
    # goal_grid = world_to_grid(goal_pos)
    # path = astar_planning(grid, start_grid, goal_grid)
    
    # if not path:
    #     print("Kein Pfad gefunden!")
    #     return False
    
    print(f"Pfad geplant mit {len(path) if 'path' in locals() else 'N'} Wegpunkten")
    
    # 3. Folge Pfad
    # for waypoint in path:
    #     while True:
    #         # Aktuelle Position
    #         current_pose = slam.get_pose()
    #         current_x, current_y, current_theta = extract_position_2d(current_pose)
    #         
    #         # Distanz zum Wegpunkt
    #         waypoint_world = grid_to_world(waypoint)
    #         dist = np.sqrt((waypoint_world[0] - current_x)**2 + 
    #                       (waypoint_world[1] - current_y)**2)
    #         
    #         if dist < 0.2:  # Wegpunkt erreicht
    #             break
    #         
    #         # DWA für lokale Steuerung
    #         obstacles = detect_obstacles_from_lidar()
    #         current_vel = (0.0, 0.0, 0.0)  # TODO: Von Roboter auslesen
    #         best_vel = dynamic_window_approach(current_vel, waypoint_world, obstacles)
    #         
    #         # Sende an Roboter
    #         bot.Move(best_vel[0], best_vel[1], best_vel[2])
    #         time.sleep(0.1)
    
    # Finale Stopp
    # bot.StopMove()
    print("Ziel erreicht!")
    return True

print("Navigations-Funktion definiert.")
print("Verwendung: navigate_to_goal(bot, slam, (8.0, 8.0))")

---
## 8. Zusammenfassung Tag 3

### Gelernte Konzepte:
✓ SLAM mit KISS-ICP  
✓ LiDAR-Integration (Livox MID-360)  
✓ Occupancy Grid Mapping  
✓ Globale Pfadplanung (A*, RRT)  
✓ Lokale Hindernisvermeidung (DWA)  
✓ RealSense RGB-D Integration  

### Wichtige Dateien:
- `live_slam.py` - SLAM mit Visualisierung
- `livox2_python.py` - LiDAR SDK Wrapper
- `jetson_realsense_stream.py` - RealSense Sender
- `receive_realsense_gst.py` - RealSense Receiver
- `mid360_config.json` - LiDAR Konfiguration

---
## 9. Hausaufgabe / Vertiefung

**Aufgabe 1:** SLAM-Mapping
- Starten Sie `live_slam.py`
- Fahren Sie den Roboter durch einen Raum
- Exportieren Sie die finale Karte

**Aufgabe 2:** A* vs. RRT Vergleich
- Implementieren Sie beide Algorithmen
- Vergleichen Sie Laufzeit und Pfad-Qualität
- Testen Sie mit verschiedenen Karten

**Aufgabe 3:** DWA Tuning
- Variieren Sie DWA-Parameter (Gewichte)
- Beobachten Sie Einfluss auf Verhalten
- Dokumentieren Sie optimale Parameter

**Bonusaufgabe:** Implementieren Sie **Hybrid A*** für glattere Pfade.

---
## 10. Nächster Tag: Wahrnehmung, Anwendung & Transfer

**Vorschau Tag 4:**
- Visuelle Wahrnehmung mit Deep Learning
- Objekt- und Umgebungserkennung
- Integration: SLAM + Perception + Control
- Projektarbeit: Real-World Challenge
- Transfer in eigene Projekte